In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")
sys.path.append("../experiments/")

In [3]:
import os

import dill
import pandas as pd
from pathlib import Path
from transformers import RobertaTokenizer, RobertaForSequenceClassification

from xbert.engine import calculate_correlation
from mnli import (read_mnli_dataset, dataset_to_input_instances, get_labels, predict,
                  MNLI_IDX2LABEL, MNLI_LABEL2IDX, OCCLUSION_STRATEGIES, GRAD_STRATEGIES, ALL_STRATEGIES)

In [4]:
from typing import List, Dict, Any


def experiment_load_relevances(experiment_dir: str,
                               relevance_filename: str = "relevances.pkl"):
    path = Path(experiment_dir)
    
    experiment_relevances = {}
    for relevance_file in path.glob(f"**/{relevance_filename}"):
        name = relevance_file.parents[0].name
        with relevance_file.open("rb") as f:
            relevances = dill.load(f)
            experiment_relevances[name] = relevances
            
    return experiment_relevances


def experiment_relevance_correlation(relevances: Dict[str, Any],
                                     strategies: List[str] = None,
                                     strategy_name_map: Dict[str, str] = None):
    strategies = strategies or ALL_STRATEGIES
    #strategies = set(strategies) & set(relevances.keys())
    strategy_name_map = strategy_name_map or {}
    
    correlations = []
    for strategy_a in strategies:
        strategy_a_mapped = strategy_name_map.get(strategy_a, strategy_a)
        
        correlation = {"Method": strategy_a_mapped}
        for strategy_b in strategies:
            strategy_b_mapped = strategy_name_map.get(strategy_b, strategy_b)
            corr = calculate_correlation(relevances[strategy_a], relevances[strategy_b])
            correlation[strategy_b_mapped] = corr
        correlations.append(correlation)
            
    df = pd.DataFrame(correlations)
    df = df.set_index("Method")
    
    df.columns = pd.MultiIndex.from_tuples([("Occlusion" if strategy in OCCLUSION_STRATEGIES else "Gradient",
                                             strategy_name_map.get(strategy, strategy))
                                            for strategy in strategies])
    
    return df

In [5]:
RESULTS_DIR = "/home/christoph/Downloads/xbert_results/"

STRATEGY_NAME_MAPPING = {
    "unk": "Unk",
    "delete": "Delete",
    "resampling": "OLM",
    "resampling_std": "OLM-S",
    "grad": "Grad.",
    "gradxinput": "Grad*Input",
    "saliency": "Sensitivity",
    "integratedgrad": "Integr. grad"
}

# MNLI

In [6]:
MNLI_RESULTS_PATH = os.path.join(RESULTS_DIR, "mnli")
mnli_experiment_relevances = experiment_load_relevances(MNLI_RESULTS_PATH)

mnli_correlation = experiment_relevance_correlation(mnli_experiment_relevances,
                                                    strategies=["unk", "delete", "resampling", "resampling_std", "grad", "gradxinput", "saliency", "integratedgrad"],
                                                    strategy_name_map=STRATEGY_NAME_MAPPING)
print(mnli_correlation.to_latex(float_format="{:0.2f}".format, multicolumn_format="c"))
mnli_correlation

/home/christoph/.pyenv/versions/3.7.4/envs/xbert/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/christoph/.pyenv/versions/3.7.4/envs/xbert/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


\begin{tabular}{lrrrrrrrr}
\toprule
{} & \multicolumn{4}{c}{Occlusion} & \multicolumn{4}{c}{Gradient} \\
{} &       UNK & Delete & LM resampling & LM resampling std &    Grad. & Grad.*input & Saliency & Integrated grad. \\
Method            &           &        &               &                   &          &             &          &                  \\
\midrule
UNK               &      1.00 &   0.73 &          0.58 &              0.32 &     0.00 &       -0.03 &     0.22 &             0.32 \\
Delete            &      0.73 &   1.00 &          0.60 &              0.32 &     0.01 &       -0.05 &     0.23 &             0.34 \\
LM resampling     &      0.58 &   0.60 &          1.00 &              0.61 &     0.00 &       -0.03 &     0.27 &             0.28 \\
LM resampling std &      0.32 &   0.32 &          0.61 &              1.00 &    -0.00 &       -0.01 &     0.35 &             0.20 \\
Grad.             &      0.00 &   0.01 &          0.00 &             -0.00 &     1.00 &       -0.00 &  

Occlusion                                            \
                        UNK    Delete LM resampling LM resampling std   
Method                                                                  
UNK                1.000000  0.734117      0.580645          0.316609   
Delete             0.734117  1.000000      0.601821          0.324969   
LM resampling      0.580645  0.601821      1.000000          0.610440   
LM resampling std  0.316609  0.324969      0.610440          1.000000   
Grad.              0.002294  0.005007      0.003561         -0.002372   
Grad.*input       -0.034539 -0.045025     -0.027769         -0.005132   
Saliency           0.222770  0.227364      0.272469          0.346814   
Integrated grad.   0.323415  0.337978      0.275429          0.197998   

                   Gradient                                         
                      Grad. Grad.*input  Saliency Integrated grad.  
Method                                                              
UNK                0.002294   -0.034539  0.222770         0.323415  
Delete             0.005007   -0.045025  0.227364         0.337978  
LM resampling      0.003561   -0.027769  0.272469         0.275429  
LM resampling std -0.002372   -0.005132  0.346814         0.197998  
Grad.              1.000000   -0.000932  0.003702         0.002953  
Grad.*input       -0.000932    1.000000  0.029389         0.002813  
Saliency           0.003702    0.029389  1.000000         0.169672  
Integrated grad.   0.002953    0.002813  0.169672         1.000000

# SST-2

In [9]:
SST2_RESULTS_PATH = os.path.join(RESULTS_DIR, "sst2")
sst2_experiment_relevances = experiment_load_relevances(SST2_RESULTS_PATH)

sst2_correlation = experiment_relevance_correlation(sst2_experiment_relevances,
                                                    strategies=["unk", "delete", "resampling", "resampling_std", "grad", "gradxinput", "saliency", "integratedgrad"],
                                                    strategy_name_map=STRATEGY_NAME_MAPPING)
print(sst2_correlation.to_latex(float_format="{:0.2f}".format, multicolumn_format="c"))
sst2_correlation

\begin{tabular}{lrrrrrrrr}
\toprule
{} & \multicolumn{4}{c}{Occlusion} & \multicolumn{4}{c}{Gradient} \\
{} &       UNK & Delete & LM resampling & LM resampling std &    Grad. & Grad.*input & Saliency & Integrated grad. \\
Method            &           &        &               &                   &          &             &          &                  \\
\midrule
UNK               &      1.00 &   0.70 &          0.60 &              0.36 &    -0.11 &        0.56 &     0.17 &             0.44 \\
Delete            &      0.70 &   1.00 &          0.68 &              0.41 &     0.06 &        0.57 &     0.26 &             0.49 \\
LM resampling     &      0.60 &   0.68 &          1.00 &              0.62 &     0.07 &        0.49 &     0.27 &             0.44 \\
LM resampling std &      0.36 &   0.41 &          0.62 &              1.00 &     0.10 &        0.28 &     0.45 &             0.27 \\
Grad.             &     -0.11 &   0.06 &          0.07 &              0.10 &     1.00 &        0.09 &  

Occlusion                                            \
                        UNK    Delete LM resampling LM resampling std   
Method                                                                  
UNK                1.000000  0.701938      0.601099          0.355563   
Delete             0.701938  1.000000      0.679008          0.408872   
LM resampling      0.601099  0.679008      1.000000          0.621485   
LM resampling std  0.355563  0.408872      0.621485          1.000000   
Grad.             -0.105752  0.059867      0.065762          0.096932   
Grad.*input        0.564114  0.569427      0.491497          0.283404   
Saliency           0.170479  0.255421      0.272031          0.447648   
Integrated grad.   0.437578  0.494065      0.437985          0.273982   

                   Gradient                                         
                      Grad. Grad.*input  Saliency Integrated grad.  
Method                                                              
UNK               -0.105752    0.564114  0.170479         0.437578  
Delete             0.059867    0.569427  0.255421         0.494065  
LM resampling      0.065762    0.491497  0.272031         0.437985  
LM resampling std  0.096932    0.283404  0.447648         0.273982  
Grad.              1.000000    0.089549  0.190294         0.031550  
Grad.*input        0.089549    1.000000  0.160369         0.444666  
Saliency           0.190294    0.160369  1.000000         0.153623  
Integrated grad.   0.031550    0.444666  0.153623         1.000000

In [9]:
COLA_RESULTS_PATH = os.path.join(RESULTS_DIR, "cola")
cola_experiment_relevances = experiment_load_relevances(COLA_RESULTS_PATH)

cola_correlation = experiment_relevance_correlation(cola_experiment_relevances,
                                                    strategies=["unk", "delete", "resampling", "resampling_std", "grad", "gradxinput", "saliency"],
                                                    #strategies=["unk", "delete", "resampling", "resampling_std", "grad", "gradxinput", "saliency", "integratedgrad"],
                                                    strategy_name_map=STRATEGY_NAME_MAPPING)
print(cola_correlation.to_latex(float_format="{:0.2f}".format, multicolumn_format="c"))
cola_correlation

NameError: name 'STRATEGY_NAME_MAPPING' is not defined